In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

# Verileri yükleyelim
train0 = pd.read_csv('/kaggle/input/academy2025/train.csv')
test0 = pd.read_csv('/kaggle/input/academy2025/testFeatures.csv')

# Tarih sütununu işleyelim
train0['tarih'] = pd.to_datetime(train0['tarih'])
test0['tarih'] = pd.to_datetime(test0['tarih'])
train0["yıl"] = train0["tarih"].dt.year
train0["ay"] = train0["tarih"].dt.month
test0["yıl"] = test0["tarih"].dt.year
test0["ay"] = test0["tarih"].dt.month
train0["haftanın_günü"] = train0["tarih"].dt.dayofweek
test0["haftanın_günü"] = test0["tarih"].dt.dayofweek

# Kategorik değişkenleri etiketleyelim
categorical_cols = ["ürün", "ürün kategorisi", "ürün üretim yeri", "market", "şehir"]
for col in categorical_cols:
    le = LabelEncoder()
    combined = pd.concat([train0[col], test0[col]])
    le.fit(combined)
    train0[col] = le.transform(train0[col])
    test0[col] = le.transform(test0[col])

# Log1p dönüşümlerini uygulayalım
train0['ürün fiyatı'] = np.log1p(train0['ürün fiyatı'])
train0['ürün besin değeri'] = np.log1p(train0['ürün besin değeri'])
test0['ürün besin değeri'] = np.log1p(test0['ürün besin değeri'])

# Özellikler ve hedef değişken
features = ["ürün", "ürün besin değeri", "ürün kategorisi", "ürün üretim yeri",
            "market", "şehir", "yıl", "ay", "haftanın_günü"]
X = train0[features]
y = train0["ürün fiyatı"]
X_test = test0[features]

# Veri setini eğitim ve doğrulama kümelerine ayırmak
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Sayısal özellikleri ölçeklendirmek
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Geliştirilmiş Derin Öğrenme Modeli Oluşturmak
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dropout(0.3), # Dropout katmanı eklendi
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3), # Bir dropout katmanı daha eklendi
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Modeli derlemek
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

# Eğitim için erken durdurma (early stopping)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Modeli eğitmek
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Modeli değerlendirmek
loss, mse = model.evaluate(X_val_scaled, y_val, verbose=0)
rmse_val_deep_learning_improved = np.sqrt(mse)
print(f"\nDoğrulama Kümesi RMSE (Geliştirilmiş Derin Öğrenme): {rmse_val_deep_learning_improved}")

# Eğitim geçmişini görselleştirmek
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Eğitim ve Doğrulama Kaybı (Geliştirilmiş Model)")
plt.show()

# Test verisi üzerinde tahmin yapmak
preds_log_deep_improved = model.predict(X_test_scaled).flatten()

# Tahminleri orijinal ölçeğe geri çevirmek
preds_deep_improved = np.expm1(preds_log_deep_improved)
preds_deep_improved[preds_deep_improved < 0] = 0

# Submission dosyası oluşturmak
submission_df_deep_improved = pd.DataFrame({'id': test0['id'], 'ürün fiyatı': preds_deep_improved})
submission_df_deep_improved.to_csv('submission_deep_learning_improved.csv', index=False)

print("\nsubmission_deep_learning_improved.csv dosyası başarıyla oluşturuldu.")

# Dosyanın gerçekten oluştuğunu kontrol etmek için (isteğe bağlı)
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))